# Bring your own tools to the enterprise gateway
In this notebook, you'll augment the existing AgentCore Gateway with your own tools of choice. This is your chance to experiment and publish reusable tools that could be made available to others via the gateway. 
- Lambda function targets
- OpenAPI schema targets
- Smithy model targets
- MCP servers targets
- Built-in templates from integration providers as targets

You can find more information for each of these target options  here: https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway-supported-targets.html 

![List](images/mcp-server-list-tools.png)

## 1. Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude Sonnet 4 model on Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit

## 2. Lambda targets: 

In the previous lab, we used the database tools from Biomni as a lambda function target for the AgentCore Gateway. You could explore reusing other tool categories from Biomni as available [here](https://github.com/snap-stanford/Biomni/tree/main/biomni/tool) . 

For example, we demonstrate literature tools from Biomni as a second target to the gateway  in the [AWS Life sciences sample agents repository](https://github.com/aws-samples/amazon-bedrock-agents-healthcare-lifesciences/tree/main/agents_catalog/28-Research-agent-biomni-gateway-tools/prerequisite/lambda-literature). 

You can add the tools to your existing Gateway as another lambda target with the help of this [notebook demonstration](https://github.com/awslabs/amazon-bedrock-agentcore-samples/blob/main/01-tutorials/02-AgentCore-gateway/01-transform-lambda-into-mcp-tools/01-gateway-target-lambda-oauth.ipynb)


## 3. MCP server as a target

Another option is to bring a MCP server as a target. AgentCore Gateway now supports onboarding pre-existing MCP server implementations as targets, in addition to Lambda and OpenAPI tools. 


#### Create a sample MCP Server and host it in AgentCore Runtime

You can replace below with your own MCP server code. This example deploys a mcp server to search PubMed. We can start by taking a look at the mcp server code. 


In [ ]:
%pycat mcp_server_pubmed.py

Deploy MCP Server to AgentCore Runtime

In [ ]:
from mcp_server_deploy import deploy_mcp_server

agent_url, agent_arn, agent_id = deploy_mcp_server()

### Add the MCP server as target for AgentCore Gateway

Configure Outbound Auth with Cognito

In [ ]:
# Configure Outbound Auth
import boto3
from utils import get_ssm_parameter

identity_client = boto3.client("bedrock-agentcore-control")
# Get credentials from SSM
runtime_client_id = get_ssm_parameter(
    "/deep-research-workshop/agentcore/machine_client_id"
)
runtime_client_secret = get_ssm_parameter(
    "/deep-research-workshop/agentcore/cognito_secret"
)
runtime_cognito_discovery_url = get_ssm_parameter(
    "/deep-research-workshop/agentcore/cognito_discovery_url"
)

cognito_provider = identity_client.create_oauth2_credential_provider(
    name="ac-gateway-mcp-server-identity",
    credentialProviderVendor="CustomOauth2",
    oauth2ProviderConfigInput={
        "customOauth2ProviderConfig": {
            "oauthDiscovery": {
                "discoveryUrl": runtime_cognito_discovery_url,
            },
            "clientId": runtime_client_id,
            "clientSecret": runtime_client_secret,
        }
    },
)
cognito_provider_arn = cognito_provider["credentialProviderArn"]
print(cognito_provider_arn)

Create the MCP server target for the gateway with Outbound OAuth

In [ ]:
import boto3

gatewayID = get_ssm_parameter("/deep-research-workshop/agentcore/gateway_id")
# Get gateway endpoint
gateway_endpoint = get_ssm_parameter("/deep-research-workshop/agentcore/gateway_url")
print(f"Gateway Endpoint - MCP URL: {gateway_endpoint}")
cognito_client = boto3.client("cognito-idp")
gateway_client = boto3.client("bedrock-agentcore-control")

resource_server_id = get_ssm_parameter(
    "/deep-research-workshop/agentcore/cognito_auth_scope"
)

print(resource_server_id)
# Extract the resource server ID
resource_server_id = resource_server_id.split("/")[
    0
]  # "default-m2m-resource-server-abc123"

# Build full scope string
runtimeScopeString = f"{resource_server_id}/invoke"
print(f"Runtime Scope String: {runtimeScopeString}")
# Create the Gateway Target
create_gateway_target_response = gateway_client.create_gateway_target(
    name="mcp-server-target",
    gatewayIdentifier=gatewayID,
    targetConfiguration={"mcp": {"mcpServer": {"endpoint": agent_url}}},
    credentialProviderConfigurations=[
        {
            "credentialProviderType": "OAUTH",
            "credentialProvider": {
                "oauthCredentialProvider": {
                    "providerArn": cognito_provider_arn,
                    "scopes": [runtimeScopeString],
                }
            },
        },
    ],
)
print(create_gateway_target_response)

In [ ]:
# Check that the Gateway target exists, and is READY
list_targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gatewayID)
print(list_targets_response)

In [ ]:
# Get gateway access token
from utils import get_gateway_access_token

jwt_token = get_gateway_access_token()
if not jwt_token:
    print("❌ Failed to get gateway access token")

### Check if the new MCP Server tools are now available through the Gateway

In [ ]:
import requests
import json


def search_tools(gateway_url, access_token, query):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}",
    }

    payload = {
        "jsonrpc": "2.0",
        "id": "search-tools-request",
        "method": "tools/call",
        "params": {
            "name": "x_amz_bedrock_agentcore_search",
            "arguments": {"query": query},
        },
    }

    response = requests.post(gateway_url, headers=headers, json=payload)
    return response.json()


results = search_tools(gateway_endpoint, jwt_token, "patient")
print(json.dumps(results, indent=2))

## Next steps : Augment your agent to access the new MCP server tools via the Gateway